<img src='./img/EU-Copernicus-EUM_3Logos.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='50%'></img>

<br>

<a href="./11_ltpy_atmospheric_composition_overview.ipynb"><< 11 - Atmospheric Composition Data - Overview</a><span style="float:right;"><a href="./211_ltpy_AC_SAF_GOME-2_L2_load_browse.ipynb">211 - AC SAF GOME-2 Level 2 Load and Browse>></a></span>

# 1.1 WEkEO Harmonized Data Access (HDA) API

[WEkEO](https://www.wekeo.eu/) is the EU Copernicus DIAS (Data and Information Access Service) reference service for environmental data, virtual processing environments and skilled user support.

WEkEO offers access to a variety of data, including different parameters sensored from Sentinel-1, Sentinel-2 and Sentinel-3. It further offers access to climate reanalysis and seasonal forecast data.

The [Harmonised Data Access (HDA) API](https://www.wekeo.eu/documentation/using_jupyter_notebooks), a REST interface, allows users to subset and download datasets from WEkEO.

This notebook is a step-by-step guide on how to search for and download data from WEkEO using the `HDA API`.
Follow the following steps:

 - [1. Search for datasets on WEkEO](#wekeo_search)
 - [2. Get the dataset's Collection ID](#wekeo_collection_id)
 - [3. Get your WEkEO API key](#wekeo_api_key)
 - [4. Initialise the WEkEO Harmonised Data Access request](#wekeo_hda_request)
 - [5. Load data descriptor file and request data](#wekeo_json)
 - [6. Download requested data](#wekeo_download)
 - [7. Unzip downloaded .zip archives](#wekeo_unzip)

#### Load required libraries

In [1]:
import os
import sys
import json
import xarray as xr
import numpy as np
from zipfile import ZipFile
import matplotlib.pyplot as plt

import glob
import matplotlib
from matplotlib import gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from IPython.core.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

#### Load helper functions

In [2]:
from ipynb.fs.full.hda_api_functions import init, get_access_token, accept_TandC, launch_query, check_job_status, get_results_list, get_download_links, download_data

<hr>

### <a id='wekeo_search'></a>1. Search for datasets on WEkEO

Under [WEkEO DATA](https://www.wekeo.eu/dataset-navigator/start), you can search all datasets available on WEkEO. As an exmaple: 29 results are returned as a list or the search term `sentinel 3 level 1`. You can click on the dataset you are interested in and you will be guided to a product description.

<br>

<div style='text-align:center;'>
<figure><img src='./img/wekeo_data_search.png' width='80%' />
    <figcaption><i>WEkEO interface to search for datasets</i></figcaption>
</figure>
</div>

### <a id='wekeo_collection_id'></a>2. Get the Collection ID 

At the end of the data description, you get additional information to the dataset, including the `Collection ID`. You need the `Collection ID` to request data from the Harmonised Data Access API. 

For `OLCI Level 1B Full Resolution - Sentinel-3` data for example, the collection ID is `EO:EUM:DAT:SENTINEL-3:OL_1_EFR___`.

<br>

<div style='text-align:center;'>
<figure><img src='./img/wekeo_collection_id.png' width='50%' />
    <figcaption><i>Additional dataset information on WEkEO</i></figcaption>
</figure>
</div>


Let's store the Collection ID as a variable to be used later.

In [169]:
collection_id = "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___"

### <a id='wekeo_api_key'></a>3. Get the WEkEO API key

In order to interact with WEkEO's Harmonised Data Access API, each user gets assigned an `API key` and `API token`. You will need the API key in order to download data in a programmatic way.

Under [https://www.wekeo.eu/api-keys](https://www.wekeo.eu/api-keys), you can retrieve your WEkEO user credentials. At the bottom of the interface, you can click on `Show hidden keys` and your `API key` is shown. <br> 
NOTE: you have to be logged in with your user account.

<br>
<div style='text-align:center;'>
<figure><img src='./img/wekeo_api_key.png' width='50%' />
    <figcaption><i>WEkEO API key interface</i></figcaption>
</figure>
</div>

<br>

You can copy paste the `API key` in the first grey box after `-H "Authorization: Basic ...` and store it as a variable called `api_key`.


In [170]:
api_key = 'REPLACE_WITH_YOUR_API_KEY'


In addition, you can also store the path to the folder, where you want to save the files after download.

In [171]:
download_dir_path = './eodata/sentinel3/olci/'
verbose = False

### <a id='wekeo_hda_request'></a>4. Initialise the Harmonised Data Access API request

In order to initialise an API request, you have to initialise a dictionary that contains information on `dataset_id`, `api_key`, `download_directory_path` and whether you wish the request to be `verbose`.

The notebook [hda_api_functions.ipynb](./hda_api_functions.ipynb) offers a set of helper functions.

Once initialised, you can request an access token with the function `get_access_token`. You might need to accept the Terms&Conditions, which you can do with the function `accept_TandC`.

In [172]:
HAPI_dict = init(collection_id, api_key, download_dir_path, verbose=verbose)
HAPI_dict = get_access_token(HAPI_dict)
HAPI_dict = accept_TandC(HAPI_dict)

Getting an access token. This token is valid for one hour only.
Success: Access token is d372e003-e2a5-347c-b4f5-adee7f4a63b0
Copernicus_General_License Terms and Conditions already accepted


### <a id='wekeo_json'></a>5. Load data descriptor file and request data

The Harmonised Data Access API can read your data request from a `JSON` file. In this JSON-based file, you can describe the dataset you are interested in downloading. The file is in principle a dictionary. Following keys can be defined:
- `datasetID`: the dataset's collection ID
- `stringChoiceValues`: type of dataset, e.g. 'Non Time Critical'
- `dataRangeSelectValues`: time period you would like to retrieve data
- `boundingBoxValues`: optional to define a subset of a global field

See an example of a `data descriptor` file [here](./olci_data_descriptor.json)

You can load the `JSON` file with `json.load()`. The function `launch_query` will launch your data request.

In [173]:
# load the query
with open('./olci_data_descriptor.json', 'r') as f:
    query = json.load(f)

# launch job
HAPI_dict = launch_query(HAPI_dict, query)


Query successfully submitted. Job ID is fd5d3235-71c9-4446-aa2e-a821f3c7f0f9


### <a id='wekeo_download'></a>6. Download requested data

The functions `check_job_status`, `get_results_list` and `get_download_links` help you to monitor the retrieval request.

As a final step, you can use the function `download_data` to initialize the data download. 

In [174]:
# wait for jobs to complete
check_job_status(HAPI_dict)

# check results
HAPI_dict = get_results_list(HAPI_dict)
HAPI_dict = get_download_links(HAPI_dict)

# download data
HAPI_dict = download_data(HAPI_dict, skip_existing=True)

The Job fd5d3235-71c9-4446-aa2e-a821f3c7f0f9 has completed
Total number of products/results: 34
File size is:   580.15 MB
[==================================================]     6.58 Mbps[  580.16] MB downloaded,  6736.66 kbps
Download complete (took 88.18680899999998 seconds)

File size is:   594.86 MB
[==================================================]     4.92 Mbps[  594.87] MB downloaded,  5040.74 kbps
Download complete (took 120.84470099999999 seconds)

File size is:   666.00 MB
[==================================================]     4.78 Mbps[  666.01] MB downloaded,  4895.47 kbps
Download complete (took 139.311395 seconds)

File size is:   600.94 MB
[================================================= ]     4.81 MbpsDownloading ./eodata/sentinel3/olci/S3B_OL_1_EFR____20190728T053753_20190728T054053_20190729T110326_0179_028_105_2160_MAR_O_NT_002.zip
File size is:   645.09 MB
[================================================= ]     4.86 MbpsDownloading ./eodata/sentinel3/olci/S3B

### <a id='wekeo_unzip'></a>7. Unzip the downloaded data archives

Sentinel-3 data are disseminated in `.zip archives`. Thus, a final step is to unzip the downloaded .zip archives. The result is a folder that ends with .SEN3 and which contains the Sentinel3 data files in `NetCDF` format.

In [64]:
def unzip_downloads():
    file_list = glob.glob('*.zip')
    print(file_list)
    for filename in file_list:
        print(filename)
        print('Unzipping file')
        with ZipFile(filename, 'r') as zipObj:
#                # Extract all the contents of zip file in current directory
           zipObj.extractall(os.path.dirname(filename))

In [65]:
unzip_downloads()

['S3B_OL_1_EFR____20190728T225445_20190728T225659_20190730T104702_0134_028_115_3780_MAR_O_NT_002.zip', 'S3A_OL_1_EFR____20190728T233418_20190728T233633_20190730T104618_0134_047_258_3780_MAR_O_NT_002.zip']
S3B_OL_1_EFR____20190728T225445_20190728T225659_20190730T104702_0134_028_115_3780_MAR_O_NT_002.zip
Unzipping file
S3A_OL_1_EFR____20190728T233418_20190728T233633_20190730T104618_0134_047_258_3780_MAR_O_NT_002.zip
Unzipping file


<br>

<a href="./11_ltpy_atmospheric_composition_overview.ipynb"><< 11 - Atmospheric Composition Data - Overview</a><span style="float:right;"><a href="./211_ltpy_AC_SAF_GOME-2_L2_load_browse.ipynb">211 - AC SAF GOME-2 Level 2 Load and Browse>></a></span>

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eo-lab/training-atmospheric-composition/">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:training@eumetsat.int>Contact</a></span></p>